To read more about the Time Series Data Stream, checkout the docs [here](https://www.elastic.co/docs/manage-data/data-store/data-streams/time-series-data-stream-tsds).

## Connect To ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'FQt-ffZfTpeh0Snf3pUAQw',
 'name': 'ae8b5b4be42b',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


## ILM policy

If you want to modify other phases of the policy, check this [example](https://www.elastic.co/docs/manage-data/data-store/data-streams/set-up-tsds#tsds-ilm-policy) in the docs.

In [2]:
from pprint import pprint

policy = {
    "phases": {
        "hot": {
            "actions": {
                "rollover": {
                    "max_age": "5m",
                }
            }
        },
        "delete": {
            "min_age": "20m",
            "actions": {
                "delete": {}
            }
        }
    }
}

response = es.ilm.put_lifecycle(name="cpu_usage_policy_v2", policy=policy)
pprint(response.body)

{'acknowledged': True}


## Index template

In [3]:
response = es.indices.put_index_template(
    name="cpu_example_template",
    index_patterns=[
        "cpu_example_template*" # Applies to any index starting with 'cpu_usage_data'
    ],
    data_stream={},
    template={
        "settings": {
            "index.mode": "time_series", # Enable time series data mode
            "index.lifecycle.name": "cpu_usage_policy_v2",  # Apply the ILM policy
        },
        "mappings":{
            "properties": {
                "@timestamp": {
                    "type": "date"
                },
                "cpu_usage":{
                    "type": "float",
                    "time_series_metric":"gauge",
                },
                "cpu_name": {
                    "type": "keyword",
                    "time_series_dimension": True
                }
            }
        }
    },
    priority=500, # A priority higher than 200 to avoid collisions with built-in templates
    meta={
        "description": "Templatefor CPU usage data",
    },
    allow_auto_create=True
)

pprint(response.body)
    

{'acknowledged': True}


## Index data

In [ ]:
import psutil
from datetime import datetime


index_alias = "cpu_example_template"
while True:
    document = {
        "@timestamp": datetime.utcnow(),
        "cpu_usage": psutil.cpu_percent(interval=0.1),
        "cpu_name": "i7-13650HX"
    }

    es.index(index=index_alias, document=document, refresh=True)